# Building a value-weighted index

In this notebook you create a **value-weighted index**. This index uses market-cap data contained in the stock exchange listings to calculate weights and 2016 stock price information. Index performance is then compared against benchmarks to evaluate the performance of the index you created.

## Table of Contents

- [Select index components & import data](#intro)

In [1]:
import pandas as pd

---
<a id='intro'></a>

## Explore and clean company listing information

To get started with the construction of a **market-value based index**, you'll work with the combined listing info for the three largest US stock exchanges, the NYSE, the NASDAQ and the AMEX.

In this and the next exercise, you will calculate **market-cap weights** for these stocks.

We have already imported pandas as pd, and loaded the listings data set with listings information from the NYSE, NASDAQ, and AMEX. The column 'Market Capitalization' is already measured in USD mn.

In [ ]:
---
<a id='intro'></a>

In [ ]:
<img src="images/ts2_001.png" alt="" style="width: 400px;"/>